In [1]:
import os
import cv2
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold


def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed = 0
seed_everything(seed)

%matplotlib inline
sns.set(style="whitegrid")
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
eyetraindf = pd.read_csv('/content/gdrive/My Drive/tesis/eyepacs/trainLabels.csv')
eyetraindf1 = eyetraindf[eyetraindf.image != '43862_left']

In [4]:
eyetraindf1

,image,level
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1
...,...,...
35121,44347_right,0
35122,44348_left,0
35123,44348_right,0
35124,44349_left,0


In [28]:
ytrain1[35124]

array([0, 1, 0, 0, 0], dtype=uint8)

In [39]:
eyetraindf1.loc[34715]

image    43862_right
level              0
Name: 34715, dtype: object

In [6]:
ytrain1 = pd.get_dummies(eyetraindf1['level']).values

In [23]:
xtrain1 = np.load('/content/gdrive/My Drive/tesis/eyepacs/xtrainchf.npy')

In [24]:
print(xtrain1.shape)
print(ytrain1.shape)


(35125, 224, 224, 3)
(35125, 5)


In [8]:
old_data = eyetraindf1

In [12]:
new_data = pd.read_csv('/content/gdrive/My Drive/tesis/aptos/train.csv')
old_data = pd.read_csv('/content/gdrive/My Drive/tesis/eyepacs/trainLabels.csv')
test_df = pd.read_csv('/content/gdrive/My Drive/tesis/aptos/test.csv')

old_data = old_data[['image', 'level']]
old_data.columns = new_data.columns
new_data['data'] = 'new'
old_data['data'] = 'old'
old_data["id_code"] = old_data["id_code"].apply(lambda x: x + ".jpeg")
new_data["id_code"] = new_data["id_code"].apply(lambda x: x + ".png")
test_df["id_code"] = test_df["id_code"].apply(lambda x: x + ".png")


diag_0 = old_data[old_data['diagnosis'] == 0].sample(frac=0.25, random_state=seed)
diag_oth = old_data[old_data['diagnosis'] != 0]
old_data = diag_0.append(diag_oth)

train_df = new_data.append(old_data)
print('Number of train samples: ', train_df.shape[0])
print('Number of test samples: ', test_df.shape[0])
display(train_df.head())


Number of train samples:  19430
Number of test samples:  1928


,id_code,diagnosis,data
0,000c1434d8d7.png,2,new
1,001639a390f0.png,4,new
2,0024cdab0c1e.png,1,new
3,002c21358ce6.png,0,new
4,005b95c28852.png,0,new


In [13]:
xoldindex = np.array(old_data.index)


In [14]:
xnewindex = np.array(new_data.index)

In [15]:
xoldindex.shape

(15768,)

In [29]:
old_data

,id_code,diagnosis,data
32132,40576_left.jpeg,0,old
26166,33112_left.jpeg,0,old
25243,31961_right.jpeg,0,old
2010,2515_left.jpeg,0,old
31965,40364_right.jpeg,0,old
...,...,...,...
35098,44323_left.jpeg,1,old
35099,44323_right.jpeg,1,old
35105,44328_right.jpeg,1,old
35112,44337_left.jpeg,1,old


In [36]:
print(xtrain2.shape)
print(ytrain2.shape)

(3662, 224, 224, 3)
(3662, 5)


In [33]:
xtrain2 = np.load('/content/gdrive/My Drive/tesis/aptos/train224.npy')

In [35]:
aptos = pd.read_csv('/content/gdrive/My Drive/tesis/aptos/train.csv')
aptos
ytrain2=pd.get_dummies(aptos['diagnosis']).values

In [40]:
x_old = np.empty((15768,224,224,3),dtype='uint8')
y_old = np.empty((15768,5),dtype='uint8')

for i in range(len(x_old)):
  if xoldindex[i] < 34714:
    x_old[i] = xtrain1[xoldindex[i]]
    y_old[i] = ytrain1[xoldindex[i]]

  elif xoldindex[i] >=  34714:
    x_old[i] = xtrain1[xoldindex[i]-1]
    y_old[i] = ytrain1[xoldindex[i]-1]

In [52]:
xtrain2.shape

(3662, 224, 224, 3)

In [53]:
xtrain1 = np.concatenate((x_old,xtrain2))

In [49]:
ytrain1 = np.concatenate((y_old,ytrain2))

In [54]:
xtrain1.shape

(19430, 224, 224, 3)

In [55]:
ytrain1.shape

(19430, 5)

In [56]:
from keras.preprocessing.image import ImageDataGenerator

In [57]:
from sklearn.model_selection import train_test_split

x_train1, x_train2, y_train1, y_train2 = train_test_split(
    xtrain1, ytrain1, 
    test_size=0.20, 
    random_state=2020
)

In [58]:
BATCH_SIZE = 8

train_datagen = ImageDataGenerator(rescale=1./ 255.,
                                   rotation_range=360,                               
                                   #shear_range=0.01,
                                   zoom_range=[0.9, 1.25],
                                   horizontal_flip=True,
                                   vertical_flip=True)

val_datagen = ImageDataGenerator(rescale=1./ 255.,
                                   rotation_range=360,                               
                                   #shear_range=0.01,
                                   zoom_range=[0.9, 1.25],
                                   horizontal_flip=True,
                                   vertical_flip=True)

In [59]:
import json
import math
import os

import cv2
from PIL import Image
import numpy as np
from keras import layers
from keras.applications import DenseNet121
#from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
import tensorflow as tf
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tqdm import tqdm

In [60]:
pip install -U efficientnet

     |████████████████████████████████| 51kB 4.3MB/s 


In [61]:
import efficientnet.keras as efn 

In [62]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from keras.layers import Input,GlobalAveragePooling2D,Dropout,Dense,Activation
from keras import applications
from keras import optimizers,Model,Sequential

In [63]:
vdata_gen =  val_datagen.flow(x_train2, y_train2, batch_size=BATCH_SIZE, seed=2020)
data_generator = train_datagen.flow(x_train1, y_train1, batch_size=BATCH_SIZE, seed=2020)

In [64]:
def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = efn.EfficientNetB5(weights='imagenet',include_top=False,input_tensor=input_tensor)
    #base_model = applications.ResNet50(weights='imagenet', include_top=False,input_tensor=input_tensor)
    #base_model.load_weights('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.5)(x)
    final_output = Dense(n_out, activation='softmax', name='final_output')(x)
    model = Model(input_tensor, final_output)
    return model

In [65]:
model = create_model(input_shape=(224, 224, 3), n_out=5)

for layer in model.layers:
    layer.trainable = True
model.summary()


115515392/115515256 [==============================] - 4s 0us/step
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 48) 1296        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 48) 192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 112, 112, 48) 0           stem_bn[0][0]                    
____________________

In [66]:
def cosine_decay_with_warmup(global_step,
                             learning_rate_base,
                             total_steps,
                             warmup_learning_rate=0.0,
                             warmup_steps=0,
                             hold_base_rate_steps=0):
    

    if total_steps < warmup_steps:
        raise ValueError('total_steps must be larger or equal to warmup_steps.')
    learning_rate = 0.5 * learning_rate_base * (1 + np.cos(
        np.pi *
        (global_step - warmup_steps - hold_base_rate_steps
         ) / float(total_steps - warmup_steps - hold_base_rate_steps)))
    if hold_base_rate_steps > 0:
        learning_rate = np.where(global_step > warmup_steps + hold_base_rate_steps,
                                 learning_rate, learning_rate_base)
    if warmup_steps > 0:
        if learning_rate_base < warmup_learning_rate:
            raise ValueError('learning_rate_base must be larger or equal to warmup_learning_rate.')
        slope = (learning_rate_base - warmup_learning_rate) / warmup_steps
        warmup_rate = slope * global_step + warmup_learning_rate
        learning_rate = np.where(global_step < warmup_steps, warmup_rate,
                                 learning_rate)
        return np.where(global_step > total_steps, 0.0, learning_rate)

class WarmUpCosineDecayScheduler(Callback):

    def __init__(self,
                 learning_rate_base,
                 total_steps,
                 global_step_init=0,
                 warmup_learning_rate=0.0,
                 warmup_steps=0,
                 hold_base_rate_steps=0,
                 verbose=0):
        

        super(WarmUpCosineDecayScheduler, self).__init__()
        self.learning_rate_base = learning_rate_base
        self.total_steps = total_steps
        self.global_step = global_step_init
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.hold_base_rate_steps = hold_base_rate_steps
        self.verbose = verbose
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.global_step = self.global_step + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        lr = cosine_decay_with_warmup(global_step=self.global_step,
                                      learning_rate_base=self.learning_rate_base,
                                      total_steps=self.total_steps,
                                      warmup_learning_rate=self.warmup_learning_rate,
                                      warmup_steps=self.warmup_steps,
                                      hold_base_rate_steps=self.hold_base_rate_steps)
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nBatch %02d: setting learning rate to %s.' % (self.global_step + 1, lr))

In [67]:
FACTOR = 4
WARMUP_LEARNING_RATE = 1e-3 * FACTOR
STEP_SIZE = len(x_train1) // BATCH_SIZE
WARMUP_EPOCHS = 5
TOTAL_STEPS_1st = WARMUP_EPOCHS * STEP_SIZE
LR_WARMUP_EPOCHS_1st = 2
WARMUP_STEPS_1st = LR_WARMUP_EPOCHS_1st * STEP_SIZE

In [68]:
cosine_lr_1st = WarmUpCosineDecayScheduler(learning_rate_base=WARMUP_LEARNING_RATE,
                                           total_steps=TOTAL_STEPS_1st,
                                           warmup_learning_rate=0.0,
                                           warmup_steps=WARMUP_STEPS_1st,
                                           hold_base_rate_steps=(2 * STEP_SIZE))

In [69]:
callback_list = [cosine_lr_1st]

In [70]:
from keras import backend as K

In [72]:
model.compile(optimizer = optimizers.SGD(lr=WARMUP_LEARNING_RATE),loss = 'categorical_crossentropy',metrics = ['accuracy'])

history_warmup = model.fit(data_generator,
                                     validation_data=vdata_gen,
                                     epochs=20,
                                     callbacks=callback_list,
                                     verbose=1)

Epoch 1/20
1943/1943 [==============================] - 342s 176ms/step - loss: 1.0603 - accuracy: 0.5665 - val_loss: 1.0377 - val_accuracy: 0.5790
Epoch 2/20
1943/1943 [==============================] - 339s 174ms/step - loss: 1.0617 - accuracy: 0.5693 - val_loss: 1.0322 - val_accuracy: 0.5818
Epoch 3/20
1943/1943 [==============================] - 339s 174ms/step - loss: 1.0660 - accuracy: 0.5647 - val_loss: 1.0267 - val_accuracy: 0.5875
Epoch 4/20
1943/1943 [==============================] - 338s 174ms/step - loss: 1.0609 - accuracy: 0.5677 - val_loss: 1.0326 - val_accuracy: 0.5903
Epoch 5/20
1943/1943 [==============================] - 339s 175ms/step - loss: 1.0609 - accuracy: 0.5640 - val_loss: 1.0326 - val_accuracy: 0.5829
Epoch 6/20
1943/1943 [==============================] - 339s 175ms/step - loss: 1.0573 - accuracy: 0.5651 - val_loss: 1.0374 - val_accuracy: 0.5803
Epoch 7/20
1943/1943 [==============================] - 339s 175ms/step - loss: 1.0635 - accuracy: 0.5650 - val_

In [ ]:
model.save('/content/gdrive/My Drive/tesis2/exp1_128_fold0.h5')

In [ ]:
model.load_weights('/content/gdrive/My Drive/tesis2/exp1_128_fold0.h5')

In [ ]:
aptosdf = pd.read_csv('/content/gdrive/My Drive/tesis/aptos/train.csv')
x_aptos = np.load('/content/gdrive/My Drive/tesis/aptos/train128.npy')

In [ ]:
y_aptos = pd.get_dummies(aptosdf['diagnosis'])

In [ ]:
x_train1, x_val1, y_train1, y_val1 = train_test_split(
    x_aptos, y_aptos, 
    test_size=0.20, 
    random_state=2020
)

In [ ]:
x_trainidrid = np.load('/content/gdrive/My Drive/tesis/idrid/idrid128.npy')
x_testidrid = np.load('/content/gdrive/My Drive/tesis/idrid/testidrid128.npy')

x_messi = np.load('/content/gdrive/My Drive/tesis/messidor/messi128.npy')
x_aptos = np.load('/content/gdrive/My Drive/tesis/aptos/train128.npy')


idridtraindf = pd.read_csv('/content/gdrive/My Drive/tesis/idrid/a. IDRiD_Disease Grading_Training Labels.csv')
idridtestdf = pd.read_csv('/content/gdrive/My Drive/tesis/idrid/b. IDRiD_Disease Grading_Testing Labels.csv')
messidf = pd.read_csv('/content/gdrive/My Drive/tesis/messidor/messidor_data.csv')
aptosdf = pd.read_csv('/content/gdrive/My Drive/tesis/aptos/train.csv')


In [ ]:
y_trainidrdid = pd.get_dummies(idridtraindf['Retinopathy grade'])
y_testdrdid = pd.get_dummies(idridtestdf['Retinopathy grade'])
y_messi = pd.get_dummies(messidf['adjudicated_dr_grade'])
y_aptos = pd.get_dummies(aptosdf['diagnosis'])

In [ ]:
xsubtrain = np.concatenate((x_aptos,x_messi))
ysubtrain = np.concatenate((y_aptos,y_messi))

In [ ]:
xsubtrain.shape

In [ ]:
x_train1messi, x_val1messi, y_train1messi, y_val1messi = train_test_split(
    xsubtrain, ysubtrain, 
    test_size=0.20, 
    random_state=2020
)

In [ ]:
x_train1 = np.concatenate((x_train1messi,x_trainidrid))
y_train1= np.concatenate((y_train1messi,y_trainidrdid))

In [ ]:
x_val1= np.concatenate((x_val1messi,x_testidrid))
y_val1= np.concatenate((y_val1messi,y_testdrdid))

In [ ]:
x = np.concatenate((x_train1,x_val1))
y = np.concatenate((y_train1,y_val1))

In [ ]:
x_train1, x_val1, y_train1, y_val1 = train_test_split(
    x, y, 
    test_size=0.20, 
    random_state=2020
)

In [ ]:
BATCH_SIZE = 32

In [ ]:
x_train1 =x_train1.astype('int8')
x_val1 =x_val1.astype('int8')
y_train1 = y_train1.astype('float32')
y_val1 = y_val1.astype('float32')

In [ ]:
data_generator = train_datagen.flow(x_train1, y_train1, batch_size=16, seed=2020)
vdata_gen =  val_datagen.flow(x_val1, y_val1, batch_size=16, seed=2020)

In [ ]:
def categorical_focal_loss(gamma=2., alpha=.25):
    
    def categorical_focal_loss_fixed(y_true, y_pred):
        
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

        
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        
        cross_entropy = -y_true * K.log(y_pred)

        
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        
        return K.mean(loss, axis=1)

    return categorical_focal_loss_fixed

In [ ]:
cosine_lr_1st = WarmUpCosineDecayScheduler(learning_rate_base=WARMUP_LEARNING_RATE,
                                           total_steps=TOTAL_STEPS_1st,
                                           warmup_learning_rate=0.0,
                                           warmup_steps=WARMUP_STEPS_1st,
                                           hold_base_rate_steps=(2 * STEP_SIZE))

callback_list = [cosine_lr_1st]

In [ ]:
for layer in model.layers:
    layer.trainable = False

for i in range(-2, 0):
    model.layers[i].trainable = True
model.summary()

In [ ]:
model.compile(optimizer = optimizers.Adam(lr=WARMUP_LEARNING_RATE),loss = categorical_focal_loss(alpha=.25, gamma=2) ,metrics = ['accuracy'])

history_warmup = model.fit(data_generator,
                                     validation_data=vdata_gen,
                                     epochs=5,
                                     callbacks=callback_list,
                                     verbose=1)

In [ ]:
TOTAL_STEPS_2nd = 20 * STEP_SIZE
LR_WARMUP_EPOCHS_2nd = 5
WARMUP_STEPS_2nd = LR_WARMUP_EPOCHS_2nd * STEP_SIZE
LEARNING_RATE = 0.001

In [ ]:
cosine_lr_2nd = WarmUpCosineDecayScheduler(learning_rate_base=LEARNING_RATE,
                                           total_steps=TOTAL_STEPS_2nd,
                                           warmup_learning_rate=0.0,
                                           warmup_steps=WARMUP_STEPS_2nd,
                                           hold_base_rate_steps=(2 * STEP_SIZE))

In [ ]:
for layer in model.layers:
    layer.trainable = True

ES_PATIENCE = 5
RLROP_PATIENCE = 3
DECAY_DROP = 0.5


es = EarlyStopping(monitor='val_loss', mode='min', patience=ES_PATIENCE, restore_best_weights=True, verbose=1)
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience=RLROP_PATIENCE, factor=DECAY_DROP, min_lr=1e-6, verbose=1)

callback_list = [es, cosine_lr_2nd]

optimizer = optimizers.Adam(lr= LEARNING_RATE)
model.compile(optimizer=optimizer, loss= categorical_focal_loss(alpha=.25, gamma=2),  metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(data_generator, epochs=20, validation_data=vdata_gen,callbacks=[callback_list])

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./ 255.,
                                   rotation_range=360,                               
                                   #shear_range=0.01,
                                   zoom_range=[0.9, 1.25],
                                   horizontal_flip=True,
                                   vertical_flip=True)

In [ ]:
vdata_gencomplete =  test_datagen.flow(x_val1, y_val1, batch_size=1, seed=2020)

In [ ]:
y_val2 = np.array([[0,0,0,0,0]])


for i in range(733):
  y_val2 =np.append(y_val2, vdata_gencomplete[i][1], axis=0)

y_val2 = np.delete(y_val2,0, axis=0)

In [ ]:
y_val12 = [np.argmax(pred1) for pred1 in y_val2]

In [ ]:
val_preds = model.predict(vdata_gencomplete,verbose = 1)

In [ ]:
val_preds1 = [np.argmax(pred) for pred in val_preds]

In [ ]:
print("Val Cohen Kappa score: %.3f" % cohen_kappa_score(val_preds1, y_val12, weights='quadratic'))

#fold 

In [ ]:
train_df = pd.read_csv('/content/gdrive/My Drive/tesis/aptos/train.csv')
test_df = pd.read_csv('/content/gdrive/My Drive/tesis/aptos/test.csv')
print('Number of train samples: ', train_df.shape[0])
print('Number of test samples: ', test_df.shape[0])
display(train_df.head())

In [ ]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

for fold_n, (trn_idx, val_idx) in enumerate(folds.split(train_df, train_df['diagnosis'])):
    print('Fold: %s, Train size: %s, Validation size %s' % 
          (fold_n, len(trn_idx), len(val_idx)))
    train_df[('fold_%s' % fold_n)] = 0
    train_df[('fold_%s' % fold_n)].loc[trn_idx] = 'train'
    train_df[('fold_%s' % fold_n)].loc[val_idx] = 'validation'
    
display(train_df.head())

In [ ]:
train_df.to_csv('5-fold.csv', index=False)
train_df.head()

In [ ]:
train_df = pd.read_csv('/content/gdrive/My Drive/tesis2/5-fold.csv')

In [ ]:
train_df.to_csv('/content/gdrive/My Drive/tesis2/5-fold.csv')

In [ ]:
train_df.head(10)

In [ ]:
train_df.info()

In [ ]:
x_train = np.load('/content/gdrive/My Drive/tesis/aptos/train224.npy')

In [ ]:
plt.imshow(x_train[0])

In [ ]:
train_df['diagnosis']

In [ ]:
fold2=pd.get_dummies(train_df['fold_1']).values

In [ ]:
fold2[0]

In [ ]:
X_train = train_df[train_df['fold_1'] == 'train']
X_val = train_df[train_df['fold_1'] == 'validation']

In [ ]:
X_train[:20]

In [ ]:
y_train1[:20]

In [ ]:
x_train_i =np.array(X_train.index)

In [ ]:
x_val_i = np.array(X_val.index)

In [ ]:
x_train_i

In [ ]:
x_train.shape

In [ ]:
x_val_i.shape

In [ ]:
y_train = pd.get_dummies(train_df['diagnosis']).values

In [ ]:
y_train.shape

In [ ]:
x_train1 = np.empty((2929,224,224,3),dtype='uint8')
x_val1 = np.empty((733,224,224,3),dtype='uint8')

y_train1 =  np.empty((2929,5),dtype='uint8')
y_val1 = np.empty((733,5),dtype='uint8')

for i in range(len(x_train_i)):
  x_train1[i] = x_train[x_train_i[i]]
  y_train1[i] = y_train[x_train_i[i]]

for j in range(len(x_val_i)):
  x_val1[j] = x_train[x_val_i[j]]
  y_val1[j] = y_train[x_val_i[j]]

In [ ]:
y_val1[:20]

In [ ]:
X_val[:20]

In [ ]:
print(x_train1.shape)
print(x_val1.shape)
print(y_train1.shape)
print(y_val1.shape)

In [ ]:
BATCH_SIZE = 16

train_datagen = ImageDataGenerator(rescale=1./ 255.,
                                   rotation_range=360,                               
                                   #shear_range=0.01,
                                   zoom_range=[0.9, 1.25],
                                   horizontal_flip=True,
                                   vertical_flip=True)

val_datagen = ImageDataGenerator(rescale=1./ 255.,
                                   rotation_range=360,                               
                                   #shear_range=0.01,
                                   zoom_range=[0.9, 1.25],
                                   horizontal_flip=True,
                                   vertical_flip=True)

test_datagen = ImageDataGenerator(rescale=1./ 255.,
                                   rotation_range=360,                               
                                   #shear_range=0.01,
                                   zoom_range=[0.9, 1.25],
                                   horizontal_flip=True,
                                   vertical_flip=True)


In [ ]:
data_generator = train_datagen.flow(x_train1, y_train1, batch_size=BATCH_SIZE, seed=2020)
vdata_gen =  test_datagen.flow(x_val1, y_val1, batch_size=BATCH_SIZE, seed=2020)

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from keras.layers import Input,GlobalAveragePooling2D,Dropout,Dense,Activation
from keras import applications
from keras import optimizers,Model,Sequential

In [ ]:
pip install -U efficientnet

In [ ]:
def cosine_decay_with_warmup(global_step,
                             learning_rate_base,
                             total_steps,
                             warmup_learning_rate=0.0,
                             warmup_steps=0,
                             hold_base_rate_steps=0):
    

    if total_steps < warmup_steps:
        raise ValueError('total_steps must be larger or equal to warmup_steps.')
    learning_rate = 0.5 * learning_rate_base * (1 + np.cos(
        np.pi *
        (global_step - warmup_steps - hold_base_rate_steps
         ) / float(total_steps - warmup_steps - hold_base_rate_steps)))
    if hold_base_rate_steps > 0:
        learning_rate = np.where(global_step > warmup_steps + hold_base_rate_steps,
                                 learning_rate, learning_rate_base)
    if warmup_steps > 0:
        if learning_rate_base < warmup_learning_rate:
            raise ValueError('learning_rate_base must be larger or equal to warmup_learning_rate.')
        slope = (learning_rate_base - warmup_learning_rate) / warmup_steps
        warmup_rate = slope * global_step + warmup_learning_rate
        learning_rate = np.where(global_step < warmup_steps, warmup_rate,
                                 learning_rate)
        return np.where(global_step > total_steps, 0.0, learning_rate)

class WarmUpCosineDecayScheduler(Callback):
    

    def __init__(self,
                 learning_rate_base,
                 total_steps,
                 global_step_init=0,
                 warmup_learning_rate=0.0,
                 warmup_steps=0,
                 hold_base_rate_steps=0,
                 verbose=0):
        

        super(WarmUpCosineDecayScheduler, self).__init__()
        self.learning_rate_base = learning_rate_base
        self.total_steps = total_steps
        self.global_step = global_step_init
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.hold_base_rate_steps = hold_base_rate_steps
        self.verbose = verbose
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.global_step = self.global_step + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        lr = cosine_decay_with_warmup(global_step=self.global_step,
                                      learning_rate_base=self.learning_rate_base,
                                      total_steps=self.total_steps,
                                      warmup_learning_rate=self.warmup_learning_rate,
                                      warmup_steps=self.warmup_steps,
                                      hold_base_rate_steps=self.hold_base_rate_steps)
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nBatch %02d: setting learning rate to %s.' % (self.global_step + 1, lr))

In [ ]:
import efficientnet.keras as efn 

In [ ]:
def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = efn.EfficientNetB5(weights='imagenet',include_top=False,input_tensor=input_tensor)
    #base_model = applications.ResNet50(weights='imagenet', include_top=False,input_tensor=input_tensor)
    #base_model.load_weights('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.5)(x)
    final_output = Dense(n_out, activation='softmax', name='final_output')(x)
    model = Model(input_tensor, final_output)
    return model

In [ ]:
model = create_model(input_shape=(224, 224, 3), n_out=5)

for layer in model.layers:
    layer.trainable = False

for i in range(-2, 0):
    model.layers[i].trainable = True
model.summary()

In [ ]:
FACTOR = 4
WARMUP_LEARNING_RATE = 1e-3 * FACTOR
STEP_SIZE = len(x_train1) // BATCH_SIZE
WARMUP_EPOCHS = 5
TOTAL_STEPS_1st = WARMUP_EPOCHS * STEP_SIZE
LR_WARMUP_EPOCHS_1st = 2
WARMUP_STEPS_1st = LR_WARMUP_EPOCHS_1st * STEP_SIZE

In [ ]:
cosine_lr_1st = WarmUpCosineDecayScheduler(learning_rate_base=WARMUP_LEARNING_RATE,
                                           total_steps=TOTAL_STEPS_1st,
                                           warmup_learning_rate=0.0,
                                           warmup_steps=WARMUP_STEPS_1st,
                                           hold_base_rate_steps=(2 * STEP_SIZE))

In [ ]:
callback_list = [cosine_lr_1st]

In [ ]:
from keras import backend as K

In [ ]:
model.compile(optimizer = optimizers.Adam(lr=WARMUP_LEARNING_RATE),loss = 'categorical_crossentropy',metrics = ['accuracy'])

history_warmup = model.fit(data_generator,
                                     validation_data=vdata_gen,
                                     epochs=5,
                                     callbacks=callback_list,
                                     verbose=1)

In [ ]:
TOTAL_STEPS_2nd = 20 * STEP_SIZE
LR_WARMUP_EPOCHS_2nd = 5
WARMUP_STEPS_2nd = LR_WARMUP_EPOCHS_2nd * STEP_SIZE
LEARNING_RATE = 0.001

In [ ]:
TOTAL_STEPS_2nd = 20 * STEP_SIZE
LR_WARMUP_EPOCHS_2nd = 5
WARMUP_STEPS_2nd = LR_WARMUP_EPOCHS_2nd * STEP_SIZE
LEARNING_RATE = 0.001

In [ ]:
cosine_lr_2nd = WarmUpCosineDecayScheduler(learning_rate_base=LEARNING_RATE,
                                           total_steps=TOTAL_STEPS_2nd,
                                           warmup_learning_rate=0.0,
                                           warmup_steps=WARMUP_STEPS_2nd,
                                           hold_base_rate_steps=(2 * STEP_SIZE))

In [ ]:
for layer in model.layers:
    layer.trainable = True

ES_PATIENCE = 5
RLROP_PATIENCE = 3
DECAY_DROP = 0.5


es = EarlyStopping(monitor='val_loss', mode='min', patience=ES_PATIENCE, restore_best_weights=True, verbose=1)
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience=RLROP_PATIENCE, factor=DECAY_DROP, min_lr=1e-6, verbose=1)

callback_list = [es, cosine_lr_2nd]

optimizer = optimizers.Adam(lr=LEARNING_RATE)
model.compile(optimizer=optimizer, loss="categorical_crossentropy",  metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(data_generator, epochs=20, validation_data=vdata_gen,callbacks=[callback_list])

In [ ]:
vdata_gencomplete =  test_datagen.flow(x_val1, y_val1, batch_size=1, seed=2020)

In [ ]:
y_val2 = np.array([[0,0,0,0,0]])


for i in range(733):
  y_val2 =np.append(y_val2, vdata_gencomplete[i][1], axis=0)

y_val2 = np.delete(y_val2,0, axis=0)

In [ ]:
y_val12 = [np.argmax(pred1) for pred1 in y_val2]

In [ ]:
val_preds = model.predict(vdata_gencomplete,verbose = 1)

In [ ]:
val_preds1 = [np.argmax(pred) for pred in val_preds]

In [ ]:
print("Val Cohen Kappa score: %.3f" % cohen_kappa_score(val_preds1, y_val12, weights='quadratic'))

In [ ]:
model.save_weights('/content/gdrive/My Drive/tesis2/effNetB5_fold1.h5')